In [1]:
!apt update > /dev/null && apt upgrade > /dev/null
!pip install kaggle > /dev/null



W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


Extracting templates from packages: 100%


[데이터셋의 주소](https://www.kaggle.com/datasets/computingvictor/transactions-fraud-datasets)

데이터셋을 다운로드 후 데이셋이 다운로드된 경로로 이동

In [2]:
import kagglehub
import os

# Download latest version
# path 현재 파일이 다운로드된 경로
path = kagglehub.dataset_download("computingvictor/transactions-fraud-datasets")

os.chdir(path) # 경로를 현재 다운로드된 파일들이 있는 경로로 변경
print(os.getcwd()) # 현재 폴더 경로; 작업 폴더 기준
print(os.listdir(os.getcwd()))

100%|██████████| 348M/348M [00:04<00:00, 89.7MB/s]

Extracting files...


/root/.cache/kagglehub/datasets/computingvictor/transactions-fraud-datasets/versions/1
['users_data.csv', 'mcc_codes.json', 'train_fraud_labels.json', 'cards_data.csv', 'transactions_data.csv']


json파일을 dataframe으로 변경하여 사용

train_fraud_labels.json 파일의 내용을 데이터프레임 파일로 변경
다른 파일들의 데이터프레임과 합치기 위해 컬럼 이름 변경

id = transactions_data.csv의 id (거래 고유 ID)

In [22]:
import pandas as pd
import json

with open('train_fraud_labels.json') as f:
    js = json.loads(f.read())
fraud = pd.DataFrame(js)
fraud = fraud.reset_index()
fraud.columns = ["id", "fraud"]
fraud.head(10)

,id,fraud
0,10649266,No
1,23410063,No
2,9316588,No
3,12478022,No
4,9558530,No
5,12532830,No
6,19526714,No
7,9906964,No
8,13224888,No
9,13749094,No


card_on_dark_web의 데이터는 cards_data.csv에 존재함
cards_data.csv파일과 병합을 위한 부분을 제외하고 제거

In [23]:
import pandas as pd

file_path = "transactions_data.csv"
trans = pd.read_csv(file_path)

columns_to_keep = ["id", "client_id"]

# 특정 컬럼만 남기기
trans = trans[columns_to_keep]

# 결과 확인
print(trans.head(10))

        id  client_id
0  7475327       1556
1  7475328        561
2  7475329       1129
3  7475331        430
4  7475332        848
5  7475333       1807
6  7475334       1556
7  7475335       1684
8  7475336        335
9  7475337        351


id를 기준으로 병합

서로 타입이 안맞기에 str로 통일 후 병합


In [25]:
fraud['id'] = fraud['id'].astype(str)
trans['id'] = trans['id'].astype(str)

merge = pd.merge(fraud, trans, on='id')
merge.head(10)

,id,fraud,client_id
0,10649266,No,11
1,23410063,No,609
2,9316588,No,1094
3,12478022,No,944
4,9558530,No,895
5,12532830,No,658
6,19526714,No,147
7,9906964,No,1895
8,13224888,No,822
9,13749094,No,1424


cards_data.csv의 파일의 client id, card_on_dark_web 컬럼만 남겨둔 후 client_id를 기준으로 병합

In [27]:
import pandas as pd

file_path = "cards_data.csv"
card = pd.read_csv(file_path)

columns_to_keep = ["client_id", "card_on_dark_web"]

# 특정 컬럼만 남기기
card = card[columns_to_keep]

# 결과 확인
print(card.head(10))

# 타입을 통일 후 통합
card['client_id'] = card['client_id'].astype(str)
merge['client_id'] = merge['client_id'].astype(str)

merge = pd.merge(card, merge, on='client_id')
merge.head(10)

   client_id card_on_dark_web
0        825               No
1        825               No
2        825               No
3        825               No
4        825               No
5       1746               No
6       1746               No
7       1746               No
8       1746               No
9       1746               No


,client_id,card_on_dark_web,id,fraud
0,825,No,16405789,No
1,825,No,23461073,No
2,825,No,14919911,No
3,825,No,23394107,No
4,825,No,18643037,No
5,825,No,17853096,No
6,825,No,19644768,No
7,825,No,8329294,No
8,825,No,11497313,No
9,825,No,9318244,No


필요없어진 client_id와 id는 제외한다.

In [28]:
columns_to_keep = ["fraud", "card_on_dark_web"]

# 특정 컬럼만 남기기
merge = merge[columns_to_keep]
merge.head(10)

,fraud,card_on_dark_web
0,No,No
1,No,No
2,No,No
3,No,No
4,No,No
5,No,No
6,No,No
7,No,No
8,No,No
9,No,No


무용지물 (미리 파악하자)

In [33]:
print(merge["fraud"].unique())
print(merge["card_on_dark_web"].unique())

['No' 'Yes']
['No']
